In [9]:
from Orch import Bijection, GemmTopology, GemmNode, ExecutionType
from dataclasses import dataclass, field
from typing import Tuple, List, Union, Optional, Dict

In [ ]:

@dataclass
class CompiledNode:
    """
    The 'Hydrated' IR Node.
    Contains concrete relative slices and strictly ordered children.
    This is the "Instruction" for the Verifier/Visualizer.
    """
    # 1. Physical Properties
    shape: Tuple[int, int, int]
    relative_slice: Tuple[slice, slice, slice] # The view relative to parent
    
    # 2. Execution Meta
    execution_type: ExecutionType 
    rank: int  # The execution rank of this node amongst its siblings
    label: str # For visualization (e.g., "Block Tile (0,1,0)")
    
    # 3. Hierarchy (Ordered by Rank, not ID)
    # List of children sorted by execution order. 
    # We use a list because order matters for Sequential nodes.
    children: List['CompiledNode'] = field(default_factory=list)
    
    # 4. Debug/Traversal
    parent: Optional['CompiledNode'] = None
    
    def is_leaf(self) -> bool:
        return len(self.children) == 0

def compile_orchestration(node: GemmNode, 
                          current_rank: int = 0,
                          parent_slice: Tuple[slice, slice, slice] = None) -> CompiledNode:
    """
    Pure Function.
    Recursively transforms a GemmNode tree into a CompiledNode tree.
    """
    
    # 1. Handle Root Case (No parent slice implies full view)
    if parent_slice is None:
        # Defaults to full slice (0:M, 0:K, 0:N)
        parent_slice = (slice(0, node.topology.shape[0]),
                        slice(0, node.topology.shape[1]),
                        slice(0, node.topology.shape[2]))

    # 2. Sort Children by Rank (The Scheduling Step)
    # We only compile children that have been explicitly set (not None)
    active_children = []
    for tile_id, child_node in node.children.items():
        if child_node is not None:
            rank = node.topology.get_rank(tile_id)
            active_children.append((rank, tile_id, child_node))
    
    # Sort by rank ascending
    active_children.sort(key=lambda x: x[0])

    # 3. Compile Current Node
    # Note: For the root, the relative slice is the "Global" slice.
    # For children, we will calculate it below.
    compiled_node = CompiledNode(
        shape=node.topology.shape,
        relative_slice=parent_slice,
        execution_type=node.execution_type,
        rank=current_rank,
        label=f"Node {node.topology.shape}"
    )

    # 4. Recursive Compilation of Children
    for rank, tile_id, child_node in active_children:
        # A. Get the slice relative to the CURRENT node
        # Your GemmTopology class already calculates this perfectly!
        rel_slice = node.topology.get_tile_slice(tile_id)
        
        # B. Recurse
        compiled_child = compile_orchestration(
            child_node, 
            current_rank=rank,
            parent_slice=rel_slice 
        )
        
        # C. Link
        compiled_child.parent = compiled_node
        compiled_node.children.append(compiled_child)

    return compiled_node

SyntaxError: invalid syntax (747751561.py, line 3)